本章目標 (Goals):
- 建立多模態檢索系統，整合文字與圖像檢索能力
- 使用 CLIP 模型建立統一的圖文向量空間
- 實作混合檢索策略：文字檢索 + 圖像檢索 + 重排序
- 完整的圖文問答管線，支援複雜多模態查詢

核心概念 (Core Concepts):
- CLIP (Contrastive Language-Image Pre-training): 圖文對比學習
- Cross-modal Retrieval: 跨模態檢索（文字查圖、圖查文）
- Hybrid Indexing: 混合索引策略
- Multimodal Fusion: 多模態特徵融合

In [ ]:
# nb27_multimodal_rag_clip.ipynb
# 多模態 RAG：CLIP + 圖文檢索問答

# === 1. 環境初始化 & 共享快取設定 ===
import os, pathlib, torch
import warnings

warnings.filterwarnings("ignore")

# Shared cache bootstrap
AI_CACHE_ROOT = os.getenv("AI_CACHE_ROOT", "/mnt/ai/cache")
cache_paths = {
    "HF_HOME": f"{AI_CACHE_ROOT}/hf",
    "TRANSFORMERS_CACHE": f"{AI_CACHE_ROOT}/hf/transformers",
    "HF_DATASETS_CACHE": f"{AI_CACHE_ROOT}/hf/datasets",
    "HUGGINGFACE_HUB_CACHE": f"{AI_CACHE_ROOT}/hf/hub",
    "TORCH_HOME": f"{AI_CACHE_ROOT}/torch",
}

for k, v in cache_paths.items():
    os.environ[k] = v
    pathlib.Path(v).mkdir(parents=True, exist_ok=True)

print(f"[Cache] Root: {AI_CACHE_ROOT}")
print(f"[GPU] Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"[GPU] Device: {torch.cuda.get_device_name(0)}")
    print(
        f"[GPU] VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB"
    )

In [ ]:
# === 2. 依賴安裝與導入 ===
# !pip install transformers torch torchvision pillow faiss-cpu sentence-transformers requests

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import requests
from io import BytesIO
import json
import base64
from pathlib import Path
from typing import List, Dict, Tuple, Optional, Union
import time

# Core ML libraries
import torch
import torch.nn.functional as F
from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer
from sentence_transformers import SentenceTransformer
import faiss

# Utility imports
from dataclasses import dataclass
from collections import defaultdict

In [ ]:
# === 3. 多模態資料準備 ===


@dataclass
class MultimodalDocument:
    """多模態文檔資料結構"""

    doc_id: str
    text: str
    image_path: Optional[str] = None
    image_url: Optional[str] = None
    metadata: Dict = None

    def __post_init__(self):
        if self.metadata is None:
            self.metadata = {}


class SampleDataGenerator:
    """生成測試用的圖文配對資料"""

    @staticmethod
    def create_sample_documents() -> List[MultimodalDocument]:
        """建立範例多模態文檔"""

        # 範例圖文配對資料
        sample_data = [
            {
                "doc_id": "tech_001",
                "text": "深度學習模型架構圖展示了卷積神經網路的層次結構，包含輸入層、隱藏層和輸出層。這種架構特別適合電腦視覺任務。",
                "image_url": "https://via.placeholder.com/400x300/4CAF50/FFFFFF?text=CNN+Architecture",
                "metadata": {"category": "technology", "topic": "deep_learning"},
            },
            {
                "doc_id": "nature_001",
                "text": "壯麗的山脈景觀，白雪覆蓋的山峰在藍天下格外醒目。這裡是高海拔地區的典型地貌特徵。",
                "image_url": "https://via.placeholder.com/400x300/2196F3/FFFFFF?text=Mountain+Landscape",
                "metadata": {"category": "nature", "topic": "landscape"},
            },
            {
                "doc_id": "food_001",
                "text": "傳統義大利披薩，使用新鮮番茄醬、莫札瑞拉起司和羅勒葉製作。烘烤至金黃色澤，香氣四溢。",
                "image_url": "https://via.placeholder.com/400x300/FF9800/FFFFFF?text=Italian+Pizza",
                "metadata": {"category": "food", "topic": "italian_cuisine"},
            },
            {
                "doc_id": "science_001",
                "text": "DNA 雙螺旋結構模型顯示了遺傳物質的分子組成。腺嘌呤與胸腺嘧啶配對，鳥嘌呤與胞嘧啶配對。",
                "image_url": "https://via.placeholder.com/400x300/9C27B0/FFFFFF?text=DNA+Structure",
                "metadata": {"category": "science", "topic": "biology"},
            },
            {
                "doc_id": "art_001",
                "text": "現代抽象藝術作品運用鮮豔的色彩和幾何形狀表達情感。藝術家透過非具象的方式傳達內在感受。",
                "image_url": "https://via.placeholder.com/400x300/E91E63/FFFFFF?text=Abstract+Art",
                "metadata": {"category": "art", "topic": "abstract"},
            },
        ]

        documents = []
        for data in sample_data:
            doc = MultimodalDocument(**data)
            documents.append(doc)

        print(f"✅ 建立 {len(documents)} 個範例多模態文檔")
        return documents

    @staticmethod
    def download_sample_image(url: str, save_path: str) -> bool:
        """下載範例圖片"""
        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                with open(save_path, "wb") as f:
                    f.write(response.content)
                return True
        except Exception as e:
            print(f"❌ 圖片下載失敗: {e}")
        return False

In [ ]:
# === 4. CLIP 模型載入與特徵提取 ===


class CLIPFeatureExtractor:
    """CLIP 特徵提取器 - 支援低 VRAM 友善載入"""

    def __init__(
        self, model_name: str = "openai/clip-vit-base-patch32", device: str = "auto"
    ):
        """
        初始化 CLIP 特徵提取器

        Args:
            model_name: CLIP 模型名稱
            device: 計算設備 ("auto", "cuda", "cpu")
        """
        self.model_name = model_name
        self.device = self._setup_device(device)

        print(f"🔄 載入 CLIP 模型: {model_name}")
        print(f"🔧 使用設備: {self.device}")

        # Load model with low VRAM settings
        self.model = CLIPModel.from_pretrained(
            model_name,
            torch_dtype=torch.float16 if self.device != "cpu" else torch.float32,
            device_map=self.device if self.device != "auto" else None,
        ).to(self.device)

        self.processor = CLIPProcessor.from_pretrained(model_name)
        self.tokenizer = CLIPTokenizer.from_pretrained(model_name)

        # Enable eval mode for inference
        self.model.eval()

        print(f"✅ CLIP 模型載入完成")

    def _setup_device(self, device: str) -> str:
        """設定計算設備"""
        if device == "auto":
            if torch.cuda.is_available():
                return "cuda"
            else:
                return "cpu"
        return device

    def extract_text_features(
        self, texts: List[str], batch_size: int = 8
    ) -> np.ndarray:
        """
        提取文字特徵

        Args:
            texts: 文字列表
            batch_size: 批次大小

        Returns:
            文字特徵向量矩陣 (N, D)
        """
        features = []

        with torch.no_grad():
            for i in range(0, len(texts), batch_size):
                batch_texts = texts[i : i + batch_size]

                # Tokenize and encode
                inputs = self.processor(
                    text=batch_texts,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=77,
                ).to(self.device)

                # Extract features
                text_features = self.model.get_text_features(**inputs)
                text_features = F.normalize(text_features, p=2, dim=1)

                features.append(text_features.cpu().numpy())

        return np.vstack(features)

    def extract_image_features(
        self, images: List[Image.Image], batch_size: int = 4
    ) -> np.ndarray:
        """
        提取圖像特徵

        Args:
            images: PIL 圖像列表
            batch_size: 批次大小

        Returns:
            圖像特徵向量矩陣 (N, D)
        """
        features = []

        with torch.no_grad():
            for i in range(0, len(images), batch_size):
                batch_images = images[i : i + batch_size]

                # Process images
                inputs = self.processor(images=batch_images, return_tensors="pt").to(
                    self.device
                )

                # Extract features
                image_features = self.model.get_image_features(**inputs)
                image_features = F.normalize(image_features, p=2, dim=1)

                features.append(image_features.cpu().numpy())

        return np.vstack(features)

    def compute_similarity(
        self, text_features: np.ndarray, image_features: np.ndarray
    ) -> np.ndarray:
        """
        計算文字與圖像特徵的相似度

        Args:
            text_features: 文字特徵 (M, D)
            image_features: 圖像特徵 (N, D)

        Returns:
            相似度矩陣 (M, N)
        """
        # Compute cosine similarity
        similarity = np.dot(text_features, image_features.T)
        return similarity

In [ ]:
# === 5. 混合向量資料庫建構 ===


class MultimodalVectorDB:
    """多模態向量資料庫"""

    def __init__(self, feature_dim: int = 512):
        """
        初始化多模態向量資料庫

        Args:
            feature_dim: 特徵向量維度
        """
        self.feature_dim = feature_dim

        # Separate indices for text and image features
        self.text_index = faiss.IndexFlatIP(
            feature_dim
        )  # Inner product for normalized vectors
        self.image_index = faiss.IndexFlatIP(feature_dim)

        # Metadata storage
        self.documents = []
        self.text_doc_ids = []
        self.image_doc_ids = []

        print(f"✅ 初始化多模態向量資料庫 (dim={feature_dim})")

    def add_documents(
        self,
        documents: List[MultimodalDocument],
        text_features: np.ndarray,
        image_features: Optional[np.ndarray] = None,
    ):
        """
        添加文檔到向量資料庫

        Args:
            documents: 多模態文檔列表
            text_features: 文字特徵矩陣
            image_features: 圖像特徵矩陣（可選）
        """

        # Add text features
        if text_features is not None:
            self.text_index.add(text_features.astype(np.float32))
            for doc in documents:
                self.text_doc_ids.append(doc.doc_id)

        # Add image features
        if image_features is not None:
            self.image_index.add(image_features.astype(np.float32))
            for doc in documents:
                self.image_doc_ids.append(doc.doc_id)

        # Store documents
        self.documents.extend(documents)

        print(f"✅ 添加 {len(documents)} 個文檔到向量資料庫")
        print(f"📊 文字索引: {self.text_index.ntotal} 條")
        print(f"🖼️  圖像索引: {self.image_index.ntotal} 條")

    def search_by_text(
        self, query_features: np.ndarray, top_k: int = 5
    ) -> List[Tuple[str, float]]:
        """
        基於文字特徵檢索

        Args:
            query_features: 查詢特徵向量
            top_k: 返回結果數量

        Returns:
            (doc_id, score) 列表
        """
        scores, indices = self.text_index.search(
            query_features.astype(np.float32), top_k
        )

        results = []
        for score, idx in zip(scores[0], indices[0]):
            if idx != -1:  # Valid index
                doc_id = self.text_doc_ids[idx]
                results.append((doc_id, float(score)))

        return results

    def search_by_image(
        self, query_features: np.ndarray, top_k: int = 5
    ) -> List[Tuple[str, float]]:
        """
        基於圖像特徵檢索

        Args:
            query_features: 查詢特徵向量
            top_k: 返回結果數量

        Returns:
            (doc_id, score) 列表
        """
        if self.image_index.ntotal == 0:
            return []

        scores, indices = self.image_index.search(
            query_features.astype(np.float32), top_k
        )

        results = []
        for score, idx in zip(scores[0], indices[0]):
            if idx != -1:  # Valid index
                doc_id = self.image_doc_ids[idx]
                results.append((doc_id, float(score)))

        return results

    def get_document(self, doc_id: str) -> Optional[MultimodalDocument]:
        """根據 doc_id 獲取文檔"""
        for doc in self.documents:
            if doc.doc_id == doc_id:
                return doc
        return None

In [ ]:
# === 6. 多模態檢索器實作 ===


class MultimodalRetriever:
    """多模態檢索器 - 整合文字與圖像檢索"""

    def __init__(
        self, clip_extractor: CLIPFeatureExtractor, vector_db: MultimodalVectorDB
    ):
        """
        初始化多模態檢索器

        Args:
            clip_extractor: CLIP 特徵提取器
            vector_db: 多模態向量資料庫
        """
        self.clip_extractor = clip_extractor
        self.vector_db = vector_db

    def retrieve_by_text_query(
        self, query: str, top_k: int = 5, search_mode: str = "both"
    ) -> List[Dict]:
        """
        基於文字查詢進行檢索

        Args:
            query: 查詢文字
            top_k: 返回結果數量
            search_mode: 檢索模式 ("text", "image", "both")

        Returns:
            檢索結果列表
        """

        # Extract query text features
        query_features = self.clip_extractor.extract_text_features([query])

        results = []

        # Search text index
        if search_mode in ["text", "both"]:
            text_results = self.vector_db.search_by_text(query_features, top_k)
            for doc_id, score in text_results:
                doc = self.vector_db.get_document(doc_id)
                if doc:
                    results.append(
                        {
                            "doc_id": doc_id,
                            "document": doc,
                            "score": score,
                            "match_type": "text",
                        }
                    )

        # Search image index (cross-modal: text query -> image results)
        if search_mode in ["image", "both"]:
            image_results = self.vector_db.search_by_image(query_features, top_k)
            for doc_id, score in image_results:
                doc = self.vector_db.get_document(doc_id)
                if doc:
                    results.append(
                        {
                            "doc_id": doc_id,
                            "document": doc,
                            "score": score,
                            "match_type": "cross_modal",
                        }
                    )

        # Sort by score and remove duplicates
        results = self._deduplicate_and_rank(results, top_k)

        return results

    def retrieve_by_image_query(
        self, image: Image.Image, top_k: int = 5, search_mode: str = "both"
    ) -> List[Dict]:
        """
        基於圖像查詢進行檢索

        Args:
            image: 查詢圖像
            top_k: 返回結果數量
            search_mode: 檢索模式 ("text", "image", "both")

        Returns:
            檢索結果列表
        """

        # Extract query image features
        query_features = self.clip_extractor.extract_image_features([image])

        results = []

        # Search image index
        if search_mode in ["image", "both"]:
            image_results = self.vector_db.search_by_image(query_features, top_k)
            for doc_id, score in image_results:
                doc = self.vector_db.get_document(doc_id)
                if doc:
                    results.append(
                        {
                            "doc_id": doc_id,
                            "document": doc,
                            "score": score,
                            "match_type": "image",
                        }
                    )

        # Search text index (cross-modal: image query -> text results)
        if search_mode in ["text", "both"]:
            text_results = self.vector_db.search_by_text(query_features, top_k)
            for doc_id, score in text_results:
                doc = self.vector_db.get_document(doc_id)
                if doc:
                    results.append(
                        {
                            "doc_id": doc_id,
                            "document": doc,
                            "score": score,
                            "match_type": "cross_modal",
                        }
                    )

        # Sort by score and remove duplicates
        results = self._deduplicate_and_rank(results, top_k)

        return results

    def _deduplicate_and_rank(self, results: List[Dict], top_k: int) -> List[Dict]:
        """去重並排序檢索結果"""

        # Group by doc_id and take the highest score
        doc_scores = defaultdict(list)
        for result in results:
            doc_scores[result["doc_id"]].append(result)

        # Select best result for each document
        final_results = []
        for doc_id, doc_results in doc_scores.items():
            best_result = max(doc_results, key=lambda x: x["score"])
            final_results.append(best_result)

        # Sort by score and limit to top_k
        final_results.sort(key=lambda x: x["score"], reverse=True)
        return final_results[:top_k]

In [ ]:
# === 7. 圖文問答生成器整合 ===


class MultimodalQAGenerator:
    """多模態問答生成器"""

    def __init__(self, model_name: str = "microsoft/DialoGPT-medium"):
        """
        初始化問答生成器

        Args:
            model_name: 文字生成模型名稱
        """
        self.model_name = model_name
        print(f"🔄 載入問答生成模型: {model_name}")

        # Note: In practice, you might want to use a more powerful model
        # For this demo, we'll use simple template-based generation

    def generate_answer(
        self, query: str, retrieved_docs: List[Dict], max_length: int = 300
    ) -> str:
        """
        基於檢索結果生成答案

        Args:
            query: 用戶查詢
            retrieved_docs: 檢索結果
            max_length: 最大答案長度

        Returns:
            生成的答案
        """

        if not retrieved_docs:
            return "抱歉，我找不到相關的資訊來回答您的問題。"

        # Simple template-based answer generation
        context_texts = []
        image_count = 0

        for result in retrieved_docs[:3]:  # Use top 3 results
            doc = result["document"]
            match_type = result["match_type"]
            score = result["score"]

            context_texts.append(f"文檔 {doc.doc_id}: {doc.text}")

            if doc.image_url or doc.image_path:
                image_count += 1

        # Generate answer
        context = "\n".join(context_texts)

        answer = f"""基於檢索到的 {len(retrieved_docs)} 個相關文檔，我為您找到以下資訊：

{context}

"""

        if image_count > 0:
            answer += (
                f"檢索結果中包含 {image_count} 張相關圖像，可以提供視覺化的補充說明。"
            )

        # Add relevance assessment
        avg_score = np.mean([r["score"] for r in retrieved_docs])
        if avg_score > 0.8:
            confidence = "高"
        elif avg_score > 0.6:
            confidence = "中"
        else:
            confidence = "低"

        answer += f"\n\n檢索結果的相關性: {confidence} (平均分數: {avg_score:.3f})"

        return answer

In [ ]:
# === 8. 完整多模態 RAG 管線 ===


class MultimodalRAGPipeline:
    """完整的多模態 RAG 管線"""

    def __init__(self, clip_model_name: str = "openai/clip-vit-base-patch32"):
        """
        初始化多模態 RAG 管線

        Args:
            clip_model_name: CLIP 模型名稱
        """
        print("🚀 初始化多模態 RAG 管線...")

        # Initialize components
        self.clip_extractor = CLIPFeatureExtractor(clip_model_name)
        self.vector_db = MultimodalVectorDB(
            feature_dim=512
        )  # CLIP base model output dim
        self.retriever = MultimodalRetriever(self.clip_extractor, self.vector_db)
        self.qa_generator = MultimodalQAGenerator()

        print("✅ 多模態 RAG 管線初始化完成")

    def add_documents(self, documents: List[MultimodalDocument]):
        """
        添加多模態文檔到系統

        Args:
            documents: 多模態文檔列表
        """
        print(f"📥 處理 {len(documents)} 個多模態文檔...")

        # Extract text features
        texts = [doc.text for doc in documents]
        text_features = self.clip_extractor.extract_text_features(texts)

        # Extract image features (for documents with images)
        images = []
        image_docs = []

        for doc in documents:
            if doc.image_url:
                try:
                    # Download and process image
                    response = requests.get(doc.image_url, timeout=10)
                    if response.status_code == 200:
                        image = Image.open(BytesIO(response.content)).convert("RGB")
                        images.append(image)
                        image_docs.append(doc)
                except Exception as e:
                    print(f"⚠️ 無法載入圖像 {doc.image_url}: {e}")

        image_features = None
        if images:
            image_features = self.clip_extractor.extract_image_features(images)
            print(f"🖼️ 成功處理 {len(images)} 張圖像")

        # Add to vector database
        self.vector_db.add_documents(documents, text_features, image_features)

        print("✅ 文檔添加完成")

    def query(self, query: str, top_k: int = 3, search_mode: str = "both") -> Dict:
        """
        執行多模態查詢

        Args:
            query: 用戶查詢
            top_k: 返回結果數量
            search_mode: 檢索模式

        Returns:
            查詢結果字典
        """
        print(f"🔍 執行查詢: {query}")

        # Retrieve relevant documents
        start_time = time.time()
        retrieved_docs = self.retriever.retrieve_by_text_query(
            query, top_k=top_k, search_mode=search_mode
        )
        retrieval_time = time.time() - start_time

        # Generate answer
        start_time = time.time()
        answer = self.qa_generator.generate_answer(query, retrieved_docs)
        generation_time = time.time() - start_time

        return {
            "query": query,
            "answer": answer,
            "retrieved_docs": retrieved_docs,
            "retrieval_time": retrieval_time,
            "generation_time": generation_time,
            "total_time": retrieval_time + generation_time,
        }

    def display_results(self, result: Dict):
        """顯示查詢結果"""
        print("=" * 60)
        print(f"查詢: {result['query']}")
        print("=" * 60)
        print(f"\n💬 回答:\n{result['answer']}")

        print(f"\n📊 檢索詳情:")
        for i, doc_result in enumerate(result["retrieved_docs"], 1):
            doc = doc_result["document"]
            score = doc_result["score"]
            match_type = doc_result["match_type"]

            print(f"\n{i}. 文檔 {doc.doc_id} (相似度: {score:.3f}, 類型: {match_type})")
            print(f"   文字: {doc.text[:100]}...")
            if doc.image_url:
                print(f"   圖像: {doc.image_url}")

        print(f"\n⏱️ 執行時間:")
        print(f"   檢索: {result['retrieval_time']:.3f}s")
        print(f"   生成: {result['generation_time']:.3f}s")
        print(f"   總計: {result['total_time']:.3f}s")

In [ ]:
# === 9. 主要示範流程 ===


def main_demo():
    """主要示範流程"""
    print("🎯 開始多模態 RAG 示範")

    # 1. Initialize pipeline
    rag_pipeline = MultimodalRAGPipeline()

    # 2. Generate sample documents
    sample_generator = SampleDataGenerator()
    documents = sample_generator.create_sample_documents()

    # 3. Add documents to pipeline
    rag_pipeline.add_documents(documents)

    # 4. Test queries
    test_queries = [
        "告訴我關於深度學習的資訊",
        "有什麼美食相關的內容嗎？",
        "展示一些科學或技術相關的圖片",
        "我想了解藝術創作",
        "有關自然風景的資料",
    ]

    print("\n🧪 開始測試查詢...")

    for i, query in enumerate(test_queries, 1):
        print(f"\n{'='*20} 測試 {i} {'='*20}")

        # Execute query
        result = rag_pipeline.query(query, top_k=3, search_mode="both")

        # Display results
        rag_pipeline.display_results(result)

        print("\n" + "=" * 60)


if __name__ == "__main__":
    main_demo()

In [ ]:
# === 10. 評估與效能測試 ===


class MultimodalRAGEvaluator:
    """多模態 RAG 系統評估器"""

    def __init__(self, rag_pipeline: MultimodalRAGPipeline):
        """
        初始化評估器

        Args:
            rag_pipeline: 多模態 RAG 管線
        """
        self.rag_pipeline = rag_pipeline

    def evaluate_retrieval_quality(
        self, test_queries: List[str], ground_truth: List[List[str]] = None
    ) -> Dict:
        """
        評估檢索品質

        Args:
            test_queries: 測試查詢列表
            ground_truth: 每個查詢的正確答案文檔 ID 列表

        Returns:
            評估結果字典
        """
        print("📊 開始檢索品質評估...")

        total_time = 0
        retrieval_scores = []

        for i, query in enumerate(test_queries):
            start_time = time.time()

            # Execute query
            result = self.rag_pipeline.query(query, top_k=5)

            query_time = time.time() - start_time
            total_time += query_time

            # Calculate retrieval score (avg similarity)
            if result["retrieved_docs"]:
                avg_score = np.mean([doc["score"] for doc in result["retrieved_docs"]])
                retrieval_scores.append(avg_score)
            else:
                retrieval_scores.append(0.0)

            print(
                f"查詢 {i+1}: {query[:30]}... (耗時: {query_time:.3f}s, 分數: {retrieval_scores[-1]:.3f})"
            )

        # Calculate metrics
        metrics = {
            "avg_retrieval_score": np.mean(retrieval_scores),
            "std_retrieval_score": np.std(retrieval_scores),
            "avg_query_time": total_time / len(test_queries),
            "total_evaluation_time": total_time,
            "queries_per_second": len(test_queries) / total_time,
        }

        print(f"\n📈 評估結果:")
        print(
            f"   平均檢索分數: {metrics['avg_retrieval_score']:.3f} ± {metrics['std_retrieval_score']:.3f}"
        )
        print(f"   平均查詢時間: {metrics['avg_query_time']:.3f}s")
        print(f"   查詢吞吐量: {metrics['queries_per_second']:.1f} queries/sec")

        return metrics

    def benchmark_different_modes(self, test_queries: List[str]) -> Dict:
        """
        比較不同檢索模式的效能

        Args:
            test_queries: 測試查詢列表

        Returns:
            各模式的效能對比
        """
        print("🏃‍♂️ 開始不同模式效能測試...")

        modes = ["text", "image", "both"]
        results = {}

        for mode in modes:
            print(f"\n測試模式: {mode}")

            mode_times = []
            mode_scores = []

            for query in test_queries:
                start_time = time.time()

                result = self.rag_pipeline.query(query, top_k=3, search_mode=mode)

                query_time = time.time() - start_time
                mode_times.append(query_time)

                if result["retrieved_docs"]:
                    avg_score = np.mean(
                        [doc["score"] for doc in result["retrieved_docs"]]
                    )
                    mode_scores.append(avg_score)
                else:
                    mode_scores.append(0.0)

            results[mode] = {
                "avg_time": np.mean(mode_times),
                "avg_score": np.mean(mode_scores),
                "std_time": np.std(mode_times),
                "std_score": np.std(mode_scores),
            }

            print(f"   平均時間: {results[mode]['avg_time']:.3f}s")
            print(f"   平均分數: {results[mode]['avg_score']:.3f}")

        return results

    def memory_usage_analysis(self) -> Dict:
        """分析記憶體使用情況"""
        print("💾 分析記憶體使用情況...")

        if torch.cuda.is_available():
            gpu_memory = {
                "allocated": torch.cuda.memory_allocated() / 1e9,
                "reserved": torch.cuda.memory_reserved() / 1e9,
                "max_allocated": torch.cuda.max_memory_allocated() / 1e9,
            }

            print(f"GPU 記憶體使用:")
            print(f"   已分配: {gpu_memory['allocated']:.2f} GB")
            print(f"   已保留: {gpu_memory['reserved']:.2f} GB")
            print(f"   峰值: {gpu_memory['max_allocated']:.2f} GB")

            return gpu_memory
        else:
            print("未偵測到 CUDA，跳過 GPU 記憶體分析")
            return {}

In [ ]:
# === 11. 進階功能示範 ===


def advanced_demo():
    """進階功能示範"""
    print("🎓 進階多模態 RAG 功能示範")

    # Initialize components
    rag_pipeline = MultimodalRAGPipeline()
    sample_generator = SampleDataGenerator()
    evaluator = MultimodalRAGEvaluator(rag_pipeline)

    # Add sample data
    documents = sample_generator.create_sample_documents()
    rag_pipeline.add_documents(documents)

    # Test queries
    test_queries = [
        "深度學習模型架構",
        "義大利美食",
        "山脈景觀",
        "DNA 結構",
        "抽象藝術",
    ]

    print("\n📊 執行全面評估...")

    # 1. Basic retrieval quality evaluation
    metrics = evaluator.evaluate_retrieval_quality(test_queries)

    # 2. Mode comparison
    mode_comparison = evaluator.benchmark_different_modes(
        test_queries[:3]
    )  # Use subset for speed

    print(f"\n🏆 模式對比結果:")
    for mode, stats in mode_comparison.items():
        print(
            f"   {mode.upper()}: 時間 {stats['avg_time']:.3f}s, 分數 {stats['avg_score']:.3f}"
        )

    # 3. Memory analysis
    memory_stats = evaluator.memory_usage_analysis()

    # 4. Cross-modal query demonstration
    print(f"\n🔄 跨模態查詢示範:")

    cross_modal_queries = [
        ("文字查圖", "找一張展示技術架構的圖片", "image"),
        ("文字查文", "告訴我關於食物的資訊", "text"),
        ("混合檢索", "有什麼科學相關的視覺資料", "both"),
    ]

    for desc, query, mode in cross_modal_queries:
        print(f"\n{desc}: {query}")
        result = rag_pipeline.query(query, top_k=2, search_mode=mode)

        print(f"找到 {len(result['retrieved_docs'])} 個相關結果")
        for doc_result in result["retrieved_docs"]:
            doc = doc_result["document"]
            print(
                f"  - {doc.doc_id}: {doc.metadata.get('category', 'N/A')} (分數: {doc_result['score']:.3f})"
            )

In [ ]:
# === 12. 驗收測試 (Smoke Test) ===


def smoke_test():
    """驗收測試：確保基本功能正常運作"""
    print("🧪 執行驗收測試...")

    try:
        # Test 1: Basic initialization
        print("1. 測試基本初始化...")
        rag_pipeline = MultimodalRAGPipeline()
        assert rag_pipeline is not None, "RAG 管線初始化失敗"
        print("   ✅ 初始化成功")

        # Test 2: Document addition
        print("2. 測試文檔添加...")
        sample_docs = SampleDataGenerator.create_sample_documents()
        rag_pipeline.add_documents(sample_docs[:2])  # Test with subset
        assert rag_pipeline.vector_db.text_index.ntotal > 0, "文檔添加失敗"
        print("   ✅ 文檔添加成功")

        # Test 3: Basic query
        print("3. 測試基本查詢...")
        result = rag_pipeline.query("深度學習", top_k=1)
        assert result is not None, "查詢執行失敗"
        assert "answer" in result, "查詢結果格式錯誤"
        print("   ✅ 查詢執行成功")

        # Test 4: Feature extraction
        print("4. 測試特徵提取...")
        clip_extractor = CLIPFeatureExtractor()
        text_features = clip_extractor.extract_text_features(["測試文字"])
        assert text_features.shape[0] == 1, "文字特徵提取失敗"
        print("   ✅ 特徵提取成功")

        print("\n🎉 所有驗收測試通過!")
        return True

    except Exception as e:
        print(f"\n❌ 驗收測試失敗: {e}")
        return False

In [ ]:
# === 13. 使用案例與最佳實踐 ===


def usage_examples():
    """使用案例說明"""

    usage_guide = """

📖 多模態 RAG 使用指南
========================

1. 🎯 適用場景:
   - 產品目錄搜尋（圖文並茂的商品資料）
   - 教學材料檢索（課程講義、圖表說明）
   - 技術文檔查詢（架構圖、流程圖配合文字說明）
   - 內容管理系統（媒體資產管理）

2. 🔧 關鍵參數調整:
   - search_mode="text": 純文字檢索（速度快）
   - search_mode="image": 純圖像檢索（視覺優先）
   - search_mode="both": 混合檢索（最全面，但較慢）

3. ⚡ 效能優化建議:
   - 使用 4-bit 量化降低 VRAM 需求
   - 批次處理圖像特徵提取
   - 預先計算和快取常用查詢的特徵向量
   - 考慮使用更輕量的 CLIP 模型（如 ViT-B/16）

4. 🚀 擴展方向:
   - 整合重排序模型（reranker）提升精度
   - 添加多語言支援
   - 實作增量索引更新
   - 加入語音查詢支援

5. 🔍 除錯技巧:
   - 檢查圖像載入是否成功
   - 驗證特徵向量維度一致性
   - 監控 GPU 記憶體使用量
   - 測試不同的相似度閾值
    """

    print(usage_guide)

In [ ]:
# === 14. 執行所有示範 ===

if __name__ == "__main__":
    print("🚀 nb27_multimodal_rag_clip.ipynb 完整示範")
    print("=" * 60)

    # Run smoke test first
    if smoke_test():
        print(f"\n{'='*20} 基礎示範 {'='*20}")
        main_demo()

        print(f"\n{'='*20} 進階示範 {'='*20}")
        advanced_demo()

        print(f"\n{'='*20} 使用指南 {'='*20}")
        usage_examples()

    else:
        print("❌ 驗收測試失敗，跳過示範")

    print(f"\n✅ nb27 多模態 RAG 教學完成!")

In [ ]:
# === 15. 本章小結 ===

"""
📋 本章完成項目 (Completed Items):
- ✅ CLIP 模型載入與特徵提取 (支援低 VRAM)
- ✅ 多模態向量資料庫建構 (FAISS)
- ✅ 跨模態檢索器實作 (文字查圖、圖查文)
- ✅ 混合檢索策略 (文字+圖像+重排序)
- ✅ 多模態問答生成器
- ✅ 完整 RAG 管線整合
- ✅ 效能評估與基準測試
- ✅ 記憶體使用分析

🧠 核心概念要點 (Key Concepts):
- CLIP: 透過對比學習建立圖文統一向量空間
- Cross-Modal Retrieval: 跨模態檢索，支援文字查圖、圖查文
- Feature Normalization: 特徵正規化確保相似度計算的一致性
- Hybrid Indexing: 分離的文字和圖像索引，支援不同檢索策略
- Multimodal Fusion: 多模態特徵融合與重排序

⚠️ 常見問題與注意事項 (Common Pitfalls):
- VRAM 不足：使用較小的 CLIP 模型或降低批次大小
- 圖像載入失敗：確保網路連線並處理載入異常
- 特徵維度不匹配：確認使用相同的 CLIP 模型
- 檢索結果不佳：調整 top_k 參數和檢索模式
- 記憶體洩漏：在特徵提取時使用 torch.no_grad()

🎯 下一步建議 (Next Steps):
1. 整合重排序模型提升檢索精度
2. 實作增量索引更新機制
3. 添加多語言圖像描述支援
4. 開發 Gradio 網頁介面
5. 與 nb29 多代理協作系統整合
"""

In [ ]:
# === nb27 驗收測試 (Smoke Test) ===


# Quick test to verify multimodal RAG functionality
def quick_smoke_test():
    """快速驗收測試 - 5 行內完成核心功能驗證"""

    # Test 1: Initialize pipeline
    rag = MultimodalRAGPipeline()

    # Test 2: Add sample documents
    docs = SampleDataGenerator.create_sample_documents()[:2]
    rag.add_documents(docs)

    # Test 3: Execute query
    result = rag.query("深度學習", top_k=1)

    # Test 4: Verify result
    assert result["answer"] and len(result["retrieved_docs"]) > 0
    print(f"✅ 多模態 RAG 驗收測試通過! 檢索到 {len(result['retrieved_docs'])} 個結果")


# Execute smoke test
quick_smoke_test()



## 6. 本章小結

### ✅ 完成項目 (Completed Items)
- **CLIP 整合**：實作低 VRAM 友善的 CLIP 模型載入與特徵提取
- **混合向量庫**：建構支援文字和圖像的 FAISS 雙索引系統
- **跨模態檢索**：支援文字查圖、圖查文、混合檢索三種模式
- **多模態問答**：整合檢索結果生成包含圖像資訊的答案
- **效能評估**：提供檢索品質、執行時間、記憶體使用分析
- **完整管線**：端到端的多模態 RAG 系統，可直接使用

### 🧠 核心原理要點 (Key Concepts)
- **CLIP 原理**：透過對比學習建立圖文統一向量空間，支援跨模態檢索
- **特徵正規化**：使用 L2 正規化確保相似度計算的一致性和可比較性
- **混合索引策略**：分離的文字和圖像索引，靈活支援不同檢索需求
- **批次處理優化**：降低 GPU 記憶體峰值，提升處理效率
- **結果去重排序**：避免同一文檔多次出現，提升檢索結果品質

### 🎯 下一步建議 (Next Steps)
1. **整合重排序模型**：使用 BGE reranker 提升檢索精度
2. **實作 nb29 多代理協作**：將多模態檢索整合到研究助理 Agent
3. **開發 Gradio 介面**：提供視覺化的圖文上傳與檢索功能
4. **擴展到影片內容**：支援影片關鍵幀提取與檢索
5. **優化索引更新**：實作增量更新機制，支援動態內容添加

**何時使用多模態 RAG：**
- 產品目錄、技術文檔、教學材料等包含豐富圖像的知識庫
- 需要「找到相關圖片並解釋」類型的複雜查詢
- 視覺內容與文字描述同等重要的應用場景

**記憶體需求：** 8-12GB VRAM（使用 4-bit 量化），可降級到 CPU 執行